#

Heatmap
Chicago 
Champaign County
(Champaign, Urbana) City

3857

In [1]:
import sys

sys.path.append('/opt/conda/share/qgis/python/plugins')
sys.path.append('/opt/conda/share/qgis/python')
sys.path.append('/opt/conda/bin')

In [2]:
import qgis

In [3]:
from qgis.core import *
from qgis.analysis import *

In [4]:
from qgis.core import QgsApplication
# QgsApplication.setPrefixPath('/opt/conda/share/qgis/', True)

In [5]:
import qgis.utils
qgis.utils.Qgis.QGIS_VERSION

'3.12.1-București'

In [6]:
# dir(qgis.analysis)

In [7]:
input_raster_day = QgsRasterLayer('data/LandScanConusDay.tif', 'raster') 
input_raster_night = QgsRasterLayer('data/LandScanConusNight.tif', 'raster') 

In [8]:
input_raster_day.width(), input_raster_day.height(), input_raster_day.crs()

(69900, 30300, <QgsCoordinateReferenceSystem: EPSG:4326>)

In [9]:
input_raster_night.width(), input_raster_night.height(), input_raster_night.crs()

(69900, 30300, <QgsCoordinateReferenceSystem: EPSG:4326>)

In [10]:
output_raster_day = 'data/tmp/Landscan_Day_non_zero.tif'
output_raster_night = 'data/tmp/Landscan_Night_non_zero.tif'

In [11]:
input_raster_day.extent()

<QgsRectangle: -125 24.25000000000001421, -66.75000023348758305 49.49999989878865847>

In [12]:
input_raster_day.setName('day')
input_raster_night.setName('night')
n_day = "\"" + input_raster_day.name() + "\""
n_night = "\"" + input_raster_night.name() + "\""

In [13]:
n_day, n_night

('"day"', '"night"')

In [14]:
formular_str = """
(({}>0)*{})/(({}>0)*1 + ({}<=0)*0)
"""

In [15]:
feedback = QgsProcessingFeedback()

In [16]:
context = QgsCoordinateTransformContext()

In [17]:
band = QgsRasterCalculatorEntry()
band.raster = input_raster_day
band.bandNumber = 1
band.ref = input_raster_day.name()

In [28]:
%%time
cal_day = QgsRasterCalculator(
    formulaString=formular_str.format(n_day,n_day,n_day,n_day),
    outputFile=output_raster_day,
    outputFormat = "GTiff",
    outputExtent = input_raster_day.extent(),
    nOutputColumns = input_raster_day.width(),
    nOutputRows = input_raster_day.height(),
    rasterEntries = [band], 
    transformContext = context
).processCalculation(feedback=feedback)

CPU times: user 2min 31s, sys: 9.52 s, total: 2min 41s
Wall time: 2min 48s


In [29]:
%%time
band = QgsRasterCalculatorEntry()
band.raster = input_raster_night
band.bandNumber = 1
band.ref = input_raster_night.name()
cal_night = QgsRasterCalculator(
    formulaString=formular_str.format(n_night,n_night,n_night,n_night),
    outputFile=output_raster_night,
    outputFormat = "GTiff",
    outputExtent = input_raster_night.extent(),
    nOutputColumns = input_raster_night.width(),
    nOutputRows = input_raster_night.height(),
    rasterEntries = [band], 
    transformContext = context
).processCalculation(feedback=feedback)

CPU times: user 2min 25s, sys: 9.14 s, total: 2min 34s
Wall time: 2min 40s


In [18]:
# help(QgsRasterCalculator)

In [19]:
a = QgsApplication.processingRegistry()
a.addProvider(QgsNativeAlgorithms())

True

In [20]:
for alg in QgsApplication.processingRegistry().algorithms():
    print("{}:{} --> {}".format(alg.provider().name(), alg.name(), alg.displayName()))

QGIS (native c++):addautoincrementalfield --> Add autoincremental field
QGIS (native c++):addfieldtoattributestable --> Add field to attributes table
QGIS (native c++):adduniquevalueindexfield --> Add unique value index field
QGIS (native c++):addxyfields --> Add X/Y fields to layer
QGIS (native c++):affinetransform --> Affine transform
QGIS (native c++):antimeridiansplit --> Geodesic line split at antimeridian
QGIS (native c++):arrayoffsetlines --> Array of offset (parallel) lines
QGIS (native c++):arraytranslatedfeatures --> Array of translated features
QGIS (native c++):aspect --> Aspect
QGIS (native c++):assignprojection --> Assign projection
QGIS (native c++):bookmarkstolayer --> Convert spatial bookmarks to layer
QGIS (native c++):boundary --> Boundary
QGIS (native c++):boundingboxes --> Bounding boxes
QGIS (native c++):buffer --> Buffer
QGIS (native c++):bufferbym --> Variable width buffer (by M value)
QGIS (native c++):calculatevectoroverlaps --> Overlap analysis
QGIS (native c

In [21]:
len(QgsApplication.processingRegistry().algorithms())

162

In [22]:
import processing
processing.algorithmHelp("native:pixelstopoints")

Raster pixels to points (native:pixelstopoints)

Creates a vector layer of points corresponding to each pixel in a raster layer.

This algorithm converts a raster layer to a vector layer, by creating point features for each individual pixel's center in the raster layer.

Any nodata pixels are skipped in the output.


----------------
Input parameters
----------------

INPUT_RASTER: Raster layer

	Parameter type:	QgsProcessingParameterRasterLayer

	Accepted data types:
		- str: layer ID
		- str: layer name
		- str: layer source
		- QgsProperty
		- QgsRasterLayer

RASTER_BAND: Band number

	Parameter type:	QgsProcessingParameterBand

	Accepted data types:
		- int
		- QgsProperty

FIELD_NAME: Field name

	Parameter type:	QgsProcessingParameterString

	Accepted data types:
		- str
		- QgsProperty

OUTPUT: Vector points

	Parameter type:	QgsProcessingParameterFeatureSink

	Accepted data types:
		- str: destination vector file, e.g. 'd:/test.shp'
		- str: 'memory:' to store result in tempora

In [30]:
parameters_ras2pt_day = {
    'FIELD_NAME' : 'VALUE', 
    'INPUT_RASTER' : 'data/LandScanConusDay.tif', 
    'OUTPUT' : 'pts/day.txt', 
    'RASTER_BAND' : 1 
}
parameters_ras2pt_night = {
    'FIELD_NAME' : 'VALUE', 
    'INPUT_RASTER' : 'data/LandScanConusNight.tif', 
    'OUTPUT' : 'pts/night.txt', 
    'RASTER_BAND' : 1
}

In [31]:
# parameters_ras2pt_day = {
#     'FIELD_NAME' : 'VALUE', 
#     'INPUT_RASTER' : 'data/IL_Day_clip_no0.tif', 
#     'OUTPUT' : 'pts/day_IL2.shp', 
#     'RASTER_BAND' : 1 
# }
# parameters_ras2pt_night = {
#     'FIELD_NAME' : 'VALUE', 
#     'INPUT_RASTER' : 'data/IL_Night_clip_no0.tif', 
#     'OUTPUT' : 'pts/night_IL2.shp', 
#     'RASTER_BAND' : 1 
# }

In [32]:
# help(processing.run)

In [33]:
%%time
processing.run("native:pixelstopoints", parameters_ras2pt_day, feedback=feedback)

CPU times: user 5min 48s, sys: 4.79 s, total: 5min 53s
Wall time: 5min 54s


{'OUTPUT': 'pts/day.txt.gxt'}

In [34]:
%%time
processing.run("native:pixelstopoints", parameters_ras2pt_night, feedback=feedback)

CPU times: user 5min 47s, sys: 3.13 s, total: 5min 50s
Wall time: 5min 52s


{'OUTPUT': 'pts/night.txt.gxt'}

{ 'CELLSIZE' : 0, 'CRS' : None, 'EXPRESSION' : '((\"LandScanConusNightIL_clip@1\">0)*\"LandScanConusNightIL_clip@1\") / ((\"LandScanConusNightIL_clip@1\">0)*1 + (\"LandScanConusNightIL_clip@1\"<=0)*0)', 'EXTENT' : None, 'LAYERS' : ['//192.168.100.3/DeptUsers/weiyec2/Downloads/LandScanIL/LandScanConusNightIL_clip.tif'], 'OUTPUT' : 'TEMPORARY_OUTPUT' }

In [35]:
help(QgsApplication)

Help on class QgsApplication in module qgis._core:

class QgsApplication(PyQt5.QtWidgets.QApplication)
 |  Extends QApplication to provide access to QGIS specific resources such
 |  as theme paths, database paths etc.
 |  
 |  This is a subclass of QApplication and should be instantiated in place of
 |  QApplication. Most methods are static in keeping with the design of QApplication.
 |  
 |  This class hides platform-specific path information and provides
 |  a portable way of referencing specific files and directories.
 |  Ideally, hard-coded paths should appear only here and not in other modules
 |  so that platform-conditional code is minimized and paths are easier
 |  to change due to centralization.
 |  
 |  Method resolution order:
 |      QgsApplication
 |      PyQt5.QtWidgets.QApplication
 |      PyQt5.QtGui.QGuiApplication
 |      PyQt5.QtCore.QCoreApplication
 |      PyQt5.QtCore.QObject
 |      sip.wrapper
 |      sip.simplewrapper
 |      builtins.object
 |  
 |  Methods d